* https://github.com/MSKCC-Computational-Pathology/MIL-nature-medicine-2019/blob/master/MIL_train.py

* https://github.com/binli123/dsmil-wsi/blob/master/attention_map.py

In [1]:
# -*- coding: utf-8 -*-
import math
import warnings

import torch
from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn.modules.module import Module
from torch.nn.modules.utils import _single, _pair, _triple, _reverse_repeat_tuple
from torch._torch_docs import reproducibility_notes

from torch.nn.common_types import _size_1_t, _size_2_t, _size_3_t
from typing import Optional, List, Tuple, Union

__all__ = ['Conv1d', 'Conv2d', 'Conv3d', 'ConvTranspose1d', 'ConvTranspose2d', 'ConvTranspose3d',
           'LazyConv1d', 'LazyConv2d', 'LazyConv3d', 'LazyConvTranspose1d', 'LazyConvTranspose2d',
           'LazyConvTranspose3d']

convolution_notes = \
    {"groups_note": r"""* :attr:`groups` controls the connections between inputs and outputs.
      :attr:`in_channels` and :attr:`out_channels` must both be divisible by
      :attr:`groups`. For example,

        * At groups=1, all inputs are convolved to all outputs.
        * At groups=2, the operation becomes equivalent to having two conv
          layers side by side, each seeing half the input channels
          and producing half the output channels, and both subsequently
          concatenated.
        * At groups= :attr:`in_channels`, each input channel is convolved with
          its own set of filters (of size
          :math:`\frac{\text{out\_channels}}{\text{in\_channels}}`).""",

        "depthwise_separable_note": r"""When `groups == in_channels` and `out_channels == K * in_channels`,
        where `K` is a positive integer, this operation is also known as a "depthwise convolution".

        In other words, for an input of size :math:`(N, C_{in}, L_{in})`,
        a depthwise convolution with a depthwise multiplier `K` can be performed with the arguments
        :math:`(C_\text{in}=C_\text{in}, C_\text{out}=C_\text{in} \times \text{K}, ..., \text{groups}=C_\text{in})`."""}  # noqa: B950





class _ConvNd(Module):

    __constants__ = ['stride', 'padding', 'dilation', 'groups',
                     'padding_mode', 'output_padding', 'in_channels',
                     'out_channels', 'kernel_size']
    __annotations__ = {'bias': Optional[torch.Tensor]}

    def _conv_forward(self, input: Tensor, weight: Tensor, bias: Optional[Tensor]) -> Tensor:
        ...

    in_channels: int
    _reversed_padding_repeated_twice: List[int]
    out_channels: int
    kernel_size: Tuple[int, ...]
    stride: Tuple[int, ...]
    padding: Union[str, Tuple[int, ...]]
    dilation: Tuple[int, ...]
    transposed: bool
    output_padding: Tuple[int, ...]
    groups: int
    padding_mode: str
    weight: Tensor
    bias: Optional[Tensor]

    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Tuple[int, ...],
                 stride: Tuple[int, ...],
                 padding: Tuple[int, ...],
                 dilation: Tuple[int, ...],
                 transposed: bool,
                 output_padding: Tuple[int, ...],
                 groups: int,
                 bias: bool,
                 padding_mode: str,
                 device=None,
                 dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        if groups <= 0:
            raise ValueError('groups must be a positive integer')
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        valid_padding_strings = {'same', 'valid'}
        if isinstance(padding, str):
            if padding not in valid_padding_strings:
                raise ValueError(
                    "Invalid padding string {!r}, should be one of {}".format(
                        padding, valid_padding_strings))
            if padding == 'same' and any(s != 1 for s in stride):
                raise ValueError("padding='same' is not supported for strided convolutions")

        valid_padding_modes = {'zeros', 'reflect', 'replicate', 'circular'}
        if padding_mode not in valid_padding_modes:
            raise ValueError("padding_mode must be one of {}, but got padding_mode='{}'".format(
                valid_padding_modes, padding_mode))
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.transposed = transposed
        self.output_padding = output_padding
        self.groups = groups
        self.padding_mode = padding_mode
        # `_reversed_padding_repeated_twice` is the padding to be passed to
        # `F.pad` if needed (e.g., for non-zero padding types that are
        # implemented as two ops: padding + conv). `F.pad` accepts paddings in
        # reverse order than the dimension.
        if isinstance(self.padding, str):
            self._reversed_padding_repeated_twice = [0, 0] * len(kernel_size)
            if padding == 'same':
                for d, k, i in zip(dilation, kernel_size,
                                   range(len(kernel_size) - 1, -1, -1)):
                    total_padding = d * (k - 1)
                    left_pad = total_padding // 2
                    self._reversed_padding_repeated_twice[2 * i] = left_pad
                    self._reversed_padding_repeated_twice[2 * i + 1] = (
                        total_padding - left_pad)
        else:
            self._reversed_padding_repeated_twice = _reverse_repeat_tuple(self.padding, 2)

        if transposed:
            self.weight = Parameter(torch.empty(
                (in_channels, out_channels // groups, *kernel_size), **factory_kwargs))
        else:
            self.weight = Parameter(torch.empty(
                (out_channels, in_channels // groups, *kernel_size), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_channels, **factory_kwargs))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(k), 1/sqrt(k)), where k = weight.size(1) * prod(*kernel_size)
        # For more details see: https://github.com/pytorch/pytorch/issues/15314#issuecomment-477448573
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            if fan_in != 0:
                bound = 1 / math.sqrt(fan_in)
                init.uniform_(self.bias, -bound, bound)

    def extra_repr(self):
        s = ('{in_channels}, {out_channels}, kernel_size={kernel_size}'
             ', stride={stride}')
        if self.padding != (0,) * len(self.padding):
            s += ', padding={padding}'
        if self.dilation != (1,) * len(self.dilation):
            s += ', dilation={dilation}'
        if self.output_padding != (0,) * len(self.output_padding):
            s += ', output_padding={output_padding}'
        if self.groups != 1:
            s += ', groups={groups}'
        if self.bias is None:
            s += ', bias=False'
        if self.padding_mode != 'zeros':
            s += ', padding_mode={padding_mode}'
        return s.format(**self.__dict__)

    def __setstate__(self, state):
        super().__setstate__(state)
        if not hasattr(self, 'padding_mode'):
            self.padding_mode = 'zeros'


class Conv2d(_ConvNd):
    __doc__ = r"""Applies a 2D convolution over an input signal composed of several input
    planes.

    In the simplest case, the output value of the layer with input size
    :math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
    can be precisely described as:

    .. math::
        \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
        \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


    where :math:`\star` is the valid 2D `cross-correlation`_ operator,
    :math:`N` is a batch size, :math:`C` denotes a number of channels,
    :math:`H` is a height of input planes in pixels, and :math:`W` is
    width in pixels.
    """ + r"""

    This module supports :ref:`TensorFloat32<tf32_on_ampere>`.

    On certain ROCm devices, when using float16 inputs this module will use :ref:`different precision<fp16_on_mi200>` for backward.

    * :attr:`stride` controls the stride for the cross-correlation, a single
      number or a tuple.

    * :attr:`padding` controls the amount of padding applied to the input. It
      can be either a string {{'valid', 'same'}} or an int / a tuple of ints giving the
      amount of implicit padding applied on both sides.

    * :attr:`dilation` controls the spacing between the kernel points; also
      known as the à trous algorithm. It is harder to describe, but this `link`_
      has a nice visualization of what :attr:`dilation` does.

    {groups_note}

    The parameters :attr:`kernel_size`, :attr:`stride`, :attr:`padding`, :attr:`dilation` can either be:

        - a single ``int`` -- in which case the same value is used for the height and width dimension
        - a ``tuple`` of two ints -- in which case, the first `int` is used for the height dimension,
          and the second `int` for the width dimension

    Note:
        {depthwise_separable_note}

    Note:
        {cudnn_reproducibility_note}

    Note:
        ``padding='valid'`` is the same as no padding. ``padding='same'`` pads
        the input so the output has the shape as the input. However, this mode
        doesn't support any stride values other than 1.

    Note:
        This module supports complex data types i.e. ``complex32, complex64, complex128``.

    Args:
        in_channels (int): Number of channels in the input image
        out_channels (int): Number of channels produced by the convolution
        kernel_size (int or tuple): Size of the convolving kernel
        stride (int or tuple, optional): Stride of the convolution. Default: 1
        padding (int, tuple or str, optional): Padding added to all four sides of
            the input. Default: 0
        padding_mode (str, optional): ``'zeros'``, ``'reflect'``,
            ``'replicate'`` or ``'circular'``. Default: ``'zeros'``
        dilation (int or tuple, optional): Spacing between kernel elements. Default: 1
        groups (int, optional): Number of blocked connections from input
            channels to output channels. Default: 1
        bias (bool, optional): If ``True``, adds a learnable bias to the
            output. Default: ``True``
    """.format(**reproducibility_notes, **convolution_notes) + r"""

    Shape:
        - Input: :math:`(N, C_{in}, H_{in}, W_{in})` or :math:`(C_{in}, H_{in}, W_{in})`
        - Output: :math:`(N, C_{out}, H_{out}, W_{out})` or :math:`(C_{out}, H_{out}, W_{out})`, where

          .. math::
              H_{out} = \left\lfloor\frac{H_{in}  + 2 \times \text{padding}[0] - \text{dilation}[0]
                        \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1\right\rfloor

          .. math::
              W_{out} = \left\lfloor\frac{W_{in}  + 2 \times \text{padding}[1] - \text{dilation}[1]
                        \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1\right\rfloor

    Attributes:
        weight (Tensor): the learnable weights of the module of shape
            :math:`(\text{out\_channels}, \frac{\text{in\_channels}}{\text{groups}},`
            :math:`\text{kernel\_size[0]}, \text{kernel\_size[1]})`.
            The values of these weights are sampled from
            :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})` where
            :math:`k = \frac{groups}{C_\text{in} * \prod_{i=0}^{1}\text{kernel\_size}[i]}`
        bias (Tensor):   the learnable bias of the module of shape
            (out_channels). If :attr:`bias` is ``True``,
            then the values of these weights are
            sampled from :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})` where
            :math:`k = \frac{groups}{C_\text{in} * \prod_{i=0}^{1}\text{kernel\_size}[i]}`

    Examples:

        >>> # With square kernels and equal stride
        >>> m = nn.Conv2d(16, 33, 3, stride=2)
        >>> # non-square kernels and unequal stride and with padding
        >>> m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
        >>> # non-square kernels and unequal stride and with padding and dilation
        >>> m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
        >>> input = torch.randn(20, 16, 50, 100)
        >>> output = m(input)

    .. _cross-correlation:
        https://en.wikipedia.org/wiki/Cross-correlation

    .. _link:
        https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: _size_2_t,
        stride: _size_2_t = 1,
        padding: Union[str, _size_2_t] = 0,
        dilation: _size_2_t = 1,
        groups: int = 1,
        bias: bool = True,
        padding_mode: str = 'zeros',  # TODO: refine this type
        device=None,
        dtype=None
    ) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        kernel_size_ = _pair(kernel_size)
        stride_ = _pair(stride)
        padding_ = padding if isinstance(padding, str) else _pair(padding)
        dilation_ = _pair(dilation)
        super().__init__(
            in_channels, out_channels, kernel_size_, stride_, padding_, dilation_,
            False, _pair(0), groups, bias, padding_mode, **factory_kwargs)

    def _conv_forward(self, input: Tensor, weight: Tensor, bias: Optional[Tensor]):
        if self.padding_mode != 'zeros':
            return F.conv2d(F.pad(input, self._reversed_padding_repeated_twice, mode=self.padding_mode),
                            weight, bias, self.stride,
                            _pair(0), self.dilation, self.groups)
        return F.conv2d(input, weight, bias, self.stride,
                        self.padding, self.dilation, self.groups)

    def forward(self, input: Tensor) -> Tensor:
        return self._conv_forward(input, self.weight, self.bias)



In [16]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


import sys
sys.path.insert(0, "helper")
sys.path.insert(0, "/helper")
sys.path.insert(0, "./helper")
sys.path.insert(0, "../helper")
print(sys.path)

# own module
from visualisation.feature_map import *

import random


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.K = 100 
        self.L = 10 # last one
        self.num_of_bases = 1 # 3rd dim
        
        self.conv1 = Conv2d(1, 32, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv2 = Conv2d(32, 64, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv3 = Conv2d(64, 128, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv1x1 = Conv2d(128, 10, kernel_size=1, stride=1, padding=0, dilation=1)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        # 4x16384
        # self.fc1 = nn.Linear(10*10*10, 10)
        #self.fc2 = nn.Linear(10, 10)
        
        #self.flat = nn.Flatten()
        
        self.fc3 = nn.Linear(10, 10)
        
        self.mish1 = torch.nn.Mish()
        self.mish2 = torch.nn.Mish()
        self.mish3 = torch.nn.Mish()
        self.mish1x1 = torch.nn.Mish()
        
        #self.sub_concept_pooling = nn.modules.MaxPool2d((self.K, 1), stride=(1,1))
        #self.instance_pooling = nn.modules.MaxPool2d((opt.num_of_bases, 1), stride=(1,1))
        
        self.flat = nn.Flatten()
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.mish1(x)
        
        x = self.conv2(x)
        x = self.mish2(x)
        
        x = self.conv3(x)
        x = self.mish3(x)
        
        x = self.conv1x1(x)
        x = self.mish1x1(x)
        
        # print(x.shape)
        
        #x = F.max_pool2d(x, 2)
        #x = self.dropout1(x)
        
        #print(x.size())
        #print(x.size()[2:])
        
        x = F.max_pool2d(x, kernel_size=x.size()[2:])
        
        x = x.reshape(x.size(0), -1)
        x = self.fc3(x)
        
        # x = self.flat(x)
        
        #x = self.fc2(x)
        
        
        #x = x.view(-1, self.L, self.K, 10)
        
        # input, kernel_size, stride, padding, dilation, ceil_mode
        #x = self.sub_concept_pooling(x).view(-1, self.L, self.num_of_bases).permute(0,2,1).unsqueeze(1)
        
        output = F.sigmoid(x)
        # x = x.reshape(x.size(0), -1)
        
        #x = torch.flatten(x, 1)
        # x = self.fc1(x)
        #x = F.relu(x)
        
        #x = self.dropout2(x)
        #x = self.fc2(x)
        #output = F.log_softmax(x, dim=1)
        
        x = self.sigmoid(x)
        
        return x



def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        
        
        data, target = data.to(device), target.unsqueeze(1) # .to(device)
        optimizer.zero_grad()
        output = model(data)
        
        target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
        
        if batch_idx == -1:
            print(data.shape) # torch.Size([4, 1, 28, 28])
            print(target)
            """
            tensor([[8],
            [7],
            [2],
            [7]])
            """
            print(target_multi_hot)
            """
            tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
            [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')
            """
        
        loss = F.binary_cross_entropy(output, target_multi_hot)
        loss.backward()
        optimizer.step()
        if batch_idx % (args.log_interval*1000) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            


def test(args, model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.unsqueeze(1) # .to(device)
            
            output = model(data)
            
            target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
            test_loss += F.binary_cross_entropy(output, target_multi_hot, reduction='mean').item()
        
            # test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.to(device).view_as(pred)).sum().item()
            
            if i == 0 and epoch % args.log_interval == 0:
            # if False: # i == 0:
                print(data.shape)
                layer = model.conv1x1 # model.conv1[0] # model.stage2[0].branch1[2] # model.fusion_layer # conv
                # run feature map
                dd = FeatureMap(model=model, layer=layer, device=device, iterations=None, lr=None)
                dd.run(data)
                dd.plot(path=f"example_results/feature_map_{epoch}.png")

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

class Parser():
    
    def __init__(self):
        self.batch_size = 128
        self.test_batch_size = 1
        self.epochs = 100
        self.lr = 0.1
        self.gamma = 0.7
        self.log_interval = 5
        self.save_model = True
        

def main():
    # Training settings
    args = Parser()
    
    if True:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if device == torch.device("cuda"):
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('example_data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('example_data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr)

    #scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader, epoch)
        #scheduler.step()
        
        
        if args.save_model and epoch % args.log_interval == 0:
            torch.save(model.state_dict(), f"example_results/mnist_cnn_{epoch}.ckpt")



main()

In [ ]:
import torch

batch_size = 3
num_classes = 10

loss_fn = torch.nn.BCELoss()

outputs_before_sigmoid = torch.randn(batch_size, num_classes)
sigmoid_outputs = torch.sigmoid(outputs_before_sigmoid)

# classes = [[2, 4, 7], [3, 6, 9]]
labels = torch.tensor([[1], [9], [4]])
# labels = labels.unsqueeze(0)
target_classes = torch.zeros(labels.size(0), 10).scatter_(1, labels, 1.)


# target_classes = torch.randint(0, 2, (batch_size, num_classes)).to(torch.float32)  # randints in [0, 2).

loss = loss_fn(sigmoid_outputs, target_classes)

# alternatively, use BCE with logits, on outputs before sigmoid.
loss_fn_2 = torch.nn.BCEWithLogitsLoss()
loss2 = loss_fn_2(outputs_before_sigmoid, target_classes)

print(sigmoid_outputs)

print(target_classes)

print(loss)
print(loss2)

# Multilabel
https://github.com/rhgao/Deep-MIML-Network/blob/master/models/MIML.py

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import numpy as np
import torch
from torch import optim
import torch.nn as nn
from .base_model import BaseModel
from . import networks
from torch.autograd import Variable
from sklearn.metrics import average_precision_score

class MIMLModel(BaseModel):
    def name(self):
        return 'MIMLModel'

    def initialize(self, opt):
        BaseModel.initialize(self, opt)
        self.isTrain = opt.isTrain

        if opt.using_multi_labels:
            self.label = self.Tensor(opt.batchSize, opt.L)
        else:
            self.label = self.Tensor(opt.batchSize)
        self.bases = self.Tensor(opt.batchSize, opt.F, opt.num_of_bases)

        self.BasesNet = networks.BasesNet(opt)
        self.sub_concept_pooling = nn.modules.MaxPool2d((opt.K, 1), stride=(1,1))
        self.instance_pooling = nn.modules.MaxPool2d((opt.num_of_bases,1), stride=(1,1))
        self.softmax = nn.Softmax(dim=-1)
        self.sigmoid = nn.Sigmoid()

        if opt.using_multi_labels:
            self.loss = nn.MultiLabelMarginLoss()
        else:
            self.loss = nn.CrossEntropyLoss()

        if(len(opt.gpu_ids)>0):
            self.BasesNet.cuda(opt.gpu_ids[0])
            self.sub_concept_pooling.cuda(opt.gpu_ids[0])
            self.instance_pooling.cuda(opt.gpu_ids[0])
            self.softmax.cuda(opt.gpu_ids[0])
            self.sigmoid.cuda(opt.gpu_ids[0])
            self.loss.cuda(opt.gpu_ids[0])

        networks.init_weights(self.BasesNet, self.opt.init_type)

        if(self.isTrain):
            if opt.using_multi_labels:
                self.optimizer = optim.Adam(list(self.BasesNet.parameters()), lr=opt.learning_rate, weight_decay=0.00001)
            else:
                self.optimizer = optim.Adam(list(self.BasesNet.parameters()), lr=opt.learning_rate, weight_decay=0.00001)
        else:
            self.BasesNet.eval()

        self.batch_loss = []
        self.batch_accuracy = []
        self.batch_ap = []

    def forward(self, input, volatile=False):
        bases = input['bases'].unsqueeze(3) #add another dimension for 2D convolution, a trick to replace fc with 1x1conv
        label = input['label']
        self.bases.resize_(bases.size()).copy_(bases)
        self.label.resize_(label.size()).copy_(label)

        #print(self.bases.size())
        # shape: (batchSize, L*K, num_of_bases)
        basesnet_output = self.BasesNet(Variable(self.bases, requires_grad=False, volatile=volatile)).view(-1, self.opt.L, self.opt.K, self.opt.num_of_bases)
        #print("sub_concept_layer_output:",basesnet_output.size())
        # shape: (batchSize, L, K, num_of_bases)
        sub_concept_pooling_output = self.sub_concept_pooling(basesnet_output).view(-1, self.opt.L, self.opt.num_of_bases).permute(0,2,1).unsqueeze(1)
        #print("sub_concept_pooling_output:",sub_concept_pooling_output.size())
        #softmax
        if self.opt.with_softmax:
            softmax_normalization_output = self.softmax(sub_concept_pooling_output)
            self.output = self.instance_pooling(softmax_normalization_output).view(-1, self.opt.L)
        else:
            self.output = self.instance_pooling(sub_concept_pooling_output).view(-1, self.opt.L)

    def getInstanceLabelRelation(self, input, volatile=True):
        bases = input['bases'].unsqueeze(3) #add another dimension for 2D convolution, a trick to replace fc with 1x1conv
        label = input['label']
        self.bases.resize_(bases.size()).copy_(bases)
        self.label.resize_(label.size()).copy_(label)

        basesnet_output = self.BasesNet(Variable(self.bases, requires_grad=False, volatile=volatile)).view(-1, self.opt.L, self.opt.K, self.opt.num_of_bases)
        instanceLabelRelation = self.sub_concept_pooling(basesnet_output).view(-1, self.opt.L, self.opt.num_of_bases).permute(0,2,1)
        if self.opt.using_multi_labels:
            self.output = self.instance_pooling(instanceLabelRelation.unsqueeze(1)).view(-1, self.opt.L)
            prediction = np.zeros(self.output.size())
            gt_label = np.zeros(self.output.size())
            max_label = self.output.max(dim=1)[1].data
            for i in range(gt_label.shape[0]):
                prediction[i,max_label[i]] = 1
            prediction[self.softmax(self.output).data.cpu().numpy() >= 0.3] = 1
            for index, x in np.ndenumerate(self.label.cpu().numpy()):
                if x == -1:
                    continue
                else:
                    gt_label[index[0],int(x)] = 1
            return self.softmax(instanceLabelRelation).data.cpu().numpy(), gt_label, prediction
        else:
            if self.opt.with_softmax:
                self.output = self.instance_pooling(self.softmax(instanceLabelRelation).unsqueeze(1)).view(-1, self.opt.L)
            else:
                self.output = self.instance_pooling(instanceLabelRelation.unsqueeze(1)).view(-1, self.opt.L)
            prediction = self.output.max(dim=1)[1].data.float()
            return self.softmax(instanceLabelRelation).data.cpu().numpy(), label, prediction

    def decrease_learning_rate(self, times, factor):
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.opt.learning_rate * pow(factor, times)
        print("current learning rate:",self.opt.learning_rate * pow(factor, times))

    def backward(self):
        if self.opt.using_multi_labels:
            label = Variable(self.label, requires_grad=False).long()
            prediction = np.zeros(self.output.size())
            #construt ground-truth label to compute mAP
            gt_label = np.zeros(self.output.size())
            max_label = self.output.max(dim=1)[1].data
            for i in range(gt_label.shape[0]):
                prediction[i,max_label[i]] = 1
            prediction[self.softmax(self.output).data.cpu().numpy() >= 0.3] = 1
            for index, x in np.ndenumerate(self.label.cpu().numpy()):
                if x == -1:
                    continue
                else:
                    gt_label[index[0],int(x)] = 1
            ap = average_precision_score(gt_label.T, prediction.T)
            self.batch_ap.append(ap)
        else:
            label = Variable(self.label, requires_grad=False).long()
            prediction = self.output.max(dim=1)[1].data.float()
            correct = (self.label.eq(prediction)).sum()
            accuracy = correct*1.0/self.label.size()[0]
            self.batch_accuracy.append(accuracy)
        loss = self.loss(self.output, label)
        self.batch_loss.append(loss.data[0]) 
        loss.backward()

    def display_train(self, writer, index):
        loss = sum(self.batch_loss)/len(self.batch_loss)
        writer.add_scalar('data/loss', loss, index)
        print('loss: ' + str(loss))
        self.batch_loss = []
        if self.opt.using_multi_labels:
            ap = sum(self.batch_ap)/len(self.batch_ap)
            writer.add_scalar('data/mAP', ap, index)
            print('mAP: ' + str(ap))
            self.batch_ap = []
        else:
            accuracy = sum(self.batch_accuracy)/len(self.batch_accuracy)
            writer.add_scalar('data/accuracy', accuracy, index)
            print('accuracy: ' + str(accuracy))
            self.batch_accuracy = []

    def display_val(self, writer, index, dataset_val):
        accuracies = []
        losses = []
        aps = []
        for i, val_data in enumerate(dataset_val):
            if i >= self.opt.validation_batches:
                break
            if self.opt.using_multi_labels:
                ap, loss = self.test_multi_label(val_data)
                aps.append(ap)
            else:
                accuracy,loss = self.test(val_data)
                accuracies.append(accuracy)
            losses.append(loss)
        if self.opt.using_multi_labels:
            ap = sum(aps) / len(aps)
            writer.add_scalar('data/val_mAP', ap, index)
            print('validation mAP is: ' + str(ap))
        else:
            accuracy = sum(accuracies)/len(accuracies)
            writer.add_scalar('data/val_accuracy', accuracy, index)
            print('validation accuracy is: ' + str(accuracy))
        loss = sum(losses)/len(losses)
        writer.add_scalar('data/val_loss', loss, index)
        print('validation loss is: ' + str(loss))

    def test(self, input):
        self.forward(input, volatile=True)
        prediction = self.output.max(dim=1)[1].data.float()
        correct = (self.label.eq(prediction)).sum()
        accuracy = correct*1.0/self.label.size()[0]
        label = Variable(self.label.long(), requires_grad=False)
        loss = self.loss(self.output, label).data.cpu().numpy()[0]
        return accuracy, loss

    def test_multi_label(self, input):
        self.forward(input, volatile=True)
        prediction = np.zeros(self.output.size())
        gt_label = np.zeros(self.output.size())
        max_label = self.output.max(dim=1)[1].data
        for i in range(gt_label.shape[0]):
            prediction[i,max_label[i]] = 1
        prediction[self.softmax(self.output).data.cpu().numpy() >= 0.3] = 1
        for index, x in np.ndenumerate(self.label.cpu().numpy()):
            if x == -1:
                continue
            else:
                gt_label[index[0],int(x)] = 1
        ap = average_precision_score(gt_label.T, prediction.T)
        label = Variable(self.label, requires_grad=False).long()
        loss = self.loss(self.output, label).data.cpu().numpy()[0]
        return ap, loss

    def optimize_parameters(self):
        self.optimizer.zero_grad()
        self.backward()
        self.optimizer.step()